In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [ ]:
merged_embeddings = np.load('/content/drive/MyDrive/merged_embeddings.npy')

In [ ]:
likes = np.load('/content/drive/MyDrive/likes.npy')

In [ ]:
merged_embeddings = merged_embeddings.reshape(51440,1,1792)

In [ ]:
train_embeddings = merged_embeddings[0:41152]
val_embeddings = merged_embeddings[41152:]

In [ ]:
train_likes = likes[0:41152]
val_likes = likes[41152:]

In [ ]:
train_likes

array([   1, 2750,   57, ...,  307,  806,   16])

In [ ]:
val_likes

array([974,   0, 300, ..., 707,   8, 908])

In [ ]:
train_labels = train_likes
val_labels = val_likes

In [ ]:
train_embeddings.shape, val_embeddings.shape

((41152, 1, 1792), (10288, 1, 1792))

In [ ]:
np.expand_dims(np.array([[1, 2, 3], [4, 5, 6]]), axis=-1)

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

In [ ]:
model  = tf.keras.Sequential([
    # tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(1792,1)),
    # tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
train_embeddings = tf.constant(np.expand_dims(np.squeeze(train_embeddings, axis=1), axis=-1))
val_embeddings = tf.constant(np.expand_dims(np.squeeze(val_embeddings, axis=1), axis=-1))
train_labels = tf.constant(train_labels)
val_labels = tf.constant(val_labels)

In [ ]:
train_embeddings.shape, val_embeddings.shape

(TensorShape([41152, 1792, 1]), TensorShape([10288, 1792, 1]))

In [ ]:
history = model.fit(train_embeddings, train_labels, validation_data=(val_embeddings, val_labels), epochs=3, batch_size=512, validation_batch_size=256)

Epoch 1/3
81/81 [==============================] - 23s 236ms/step - loss: 24780052.0000 - root_mean_squared_error: 4977.9565 - val_loss: 14342334.0000 - val_root_mean_squared_error: 3787.1274
Epoch 2/3
81/81 [==============================] - 19s 230ms/step - loss: 24366782.0000 - root_mean_squared_error: 4936.2720 - val_loss: 14086069.0000 - val_root_mean_squared_error: 3753.1411
Epoch 3/3
81/81 [==============================] - 18s 229ms/step - loss: 24292898.0000 - root_mean_squared_error: 4928.7827 - val_loss: 14085980.0000 - val_root_mean_squared_error: 3753.1294


In [ ]:
# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx]), torch.tensor(self.labels[idx])

# Define your regression model
class ShallowNetwork(nn.Module):
    def __init__(self, input_size, num_filters, kernel_size):
        super(ShallowNetwork, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=input_size, out_channels=num_filters,
                                kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 1)


    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)

        # Global Max Pooling
        x = self.global_pooling(x)
        x = x.view(x.size(0), -1)  # Flatten the output

        # Fully connected layers
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)


# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, criterion, optimizer on GPU
model = ShallowNetwork(input_size=512, kernel_size=3, num_filters=64).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Prepare your data (embeddings and labels) for training and validation on GPU
# Replace embeddings_train, labels_train, embeddings_val, labels_val with your data
embeddings_train = train_embeddings
labels_train = train_labels
embeddings_val = val_embeddings
labels_val = val_labels

# Move data to GPU
embeddings_train = torch.tensor(embeddings_train).to(device)
labels_train = torch.tensor(labels_train).to(device)
embeddings_val = torch.tensor(embeddings_val).to(device)
labels_val = torch.tensor(labels_val).to(device)

# Create DataLoader for training and validation
train_dataset = CustomDataset(embeddings_train, labels_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = CustomDataset(embeddings_val, labels_val)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training and validation loop with RMSE and loss calculation on GPU
epochs = 10
for epoch in range(epochs):
    # Training loop
    model.train()
    total_loss_train = 0.0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())  # Assuming inputs are float type
        loss = criterion(outputs, targets.unsqueeze(1).float())  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()
        total_loss_train += loss.item() * inputs.size(0)  # Accumulate the total loss

    epoch_loss_train = total_loss_train / len(train_dataloader.dataset)
    rmse_train = torch.sqrt(torch.tensor(epoch_loss_train, device=device))  # Calculate RMSE for training

    # Validation loop
    model.eval()
    total_loss_val = 0.0
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs.float())  # Assuming inputs are float type
            loss = criterion(outputs, targets.unsqueeze(1).float())  # Calculate loss
            total_loss_val += loss.item() * inputs.size(0)  # Accumulate the total loss

    epoch_loss_val = total_loss_val / len(val_dataloader.dataset)
    rmse_val = torch.sqrt(torch.tensor(epoch_loss_val, device=device))  # Calculate RMSE for validation

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss_train:.4f}, Train RMSE: {rmse_train:.4f}, \
          Val Loss: {epoch_loss_val:.4f}, Val RMSE: {rmse_val:.4f}")


In [ ]:
# Tetsing
k = nn.Conv1d(1,64,kernel_size=5)
input = torch.randn(64, 1, 512)
print(k(input).shape) # -> torch.Size([1, 64, 736])

torch.Size([64, 64, 508])


In [ ]:
input

tensor([[[-1.0701e+00, -4.1138e-02, -3.8949e-02,  3.7355e-01,  4.2960e-01,
          -7.8454e-01, -3.8276e-01,  9.5227e-01, -5.5946e-01,  1.3074e+00,
          -9.0360e-01,  5.1104e-01,  1.0577e+00, -7.0164e-01, -3.7643e-01,
          -1.0538e+00, -3.7156e-01, -3.7437e-01, -1.1957e+00, -8.4552e-01,
          -9.7767e-01,  6.1196e-02,  2.8820e+00,  3.3508e-01, -1.1501e+00,
           1.2041e+00, -8.6040e-01,  1.3077e+00, -1.1140e+00,  2.0064e-01,
           1.1927e+00, -8.3233e-01,  8.4071e-01, -6.2430e-02,  7.7285e-01,
           1.4545e+00,  4.8790e-01, -5.1689e-01, -3.5740e-01,  1.7161e+00,
           1.3773e+00,  9.0333e-01, -1.7872e-01,  4.5178e-01, -8.1685e-01,
          -1.6507e-01,  4.6807e-01, -9.1544e-01, -7.8223e-01, -6.7237e-01,
          -3.2841e+00, -6.9346e-01,  2.3468e-01,  1.7971e-01,  6.0856e-01,
           2.2847e-01,  9.7044e-01, -1.2593e-01, -5.6233e-01, -8.6351e-01,
          -7.6913e-01, -1.2517e+00, -1.1104e+00, -1.5638e+00,  1.4023e+00,
          -6.4234e-01,  7

In [ ]:
embeddings_train.shape

torch.Size([208801, 1, 512])

In [ ]:
k(input)

tensor([[[ 0.1971,  0.5489,  0.6214,  ..., -0.0104,  0.7965, -0.4791],
         [ 0.0739, -0.0224,  0.6053,  ..., -0.2242,  0.3812,  0.3371],
         [-0.4516, -0.3743, -0.0386,  ..., -0.6171,  0.2513, -0.9311],
         ...,
         [ 0.0981, -0.4668, -0.2003,  ..., -0.9016,  0.0403,  0.1120],
         [-0.0456, -0.0982,  0.5883,  ..., -0.0868,  0.2144,  0.3239],
         [ 0.6192, -0.0426,  0.3622,  ...,  0.1039,  0.8762,  0.2422]]],
       grad_fn=<ConvolutionBackward0>)